<a href="https://colab.research.google.com/github/cengaiyeung/20MA573/blob/master/src/hw10_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW 10

Consider 2-d PDE
$$\frac{1}{2} \Delta v(x) - v(x) + x_{1}^2 + x_{2}^2 - x_{1} - x_{2} - \frac{3}{2} = 0, x \in O=(0,1)^2$$
with its boundary data
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2, x \in O.$$
The exact solution is 
$$v(x) = (x_{1} - \frac{1}{2})^2 + (x_{2} - \frac{1}{2})^2.$$
- Idendify $MRP$ with $CFD$ in the form of , 
$$v(x) = \gamma \{ \ell ^h (x) + \sum_{i=1}^{d} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \}.$$
- For h=1/8, compute CFD solution by value iteration.
- For h=1/8, compute CFD solution by Monte-Carlo method.
- For h=1/8, compute CFD solution by TD method.
- Compare above three methodsand conclude your observations.

- Idendify $MRP$ with $CFD$ in the form of , 
$$v(x) = \gamma \{ \ell ^h (x) + \sum_{i=1}^{d} p^h (x + he_{i}|x)v(x + he_{i}) + p^h(x-he_{i}|x)v(x - he_{i}) \}.$$

# Pf:
In last class, we have proved that 
$$\gamma = \frac{2}{2+h^2}, $$
$$\ell ^{h}(x) = \frac{h^2}{2}(x_{1}^2 + x_{2}^2 - x_{1} - x_{2} - \frac{3}{2}),$$
$$p^h(x \pm he_{i}|x) = \frac{1}{4}$$
Then we know the transition probability is 
$$p(s, s^{'})=
\begin{cases}
\frac{1}{4}, & \|s - s^{'}\| = 1 \\
0,& otherwise.
\end{cases}$$

And the reward function is

$$R(s) = \gamma l^{h}(x) = \frac{h^{2}}{2+h^{2}} [(x_{1} - \frac{1}{2})^{2} + (x_{2} - \frac{1}{2})^{2} -2].$$


- For h=1/8, compute CFD solution by value iteration.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

In [0]:

def gamma(N):
  h = 1/N
  gamma = 2/(2+h**2)
  return gamma

In [4]:
#Test gamma
print("When N = 8, the gamma is", gamma(8))

When N = 8, the gamma is 0.9922480620155039


In [0]:
def l(N):
  lh_matrix = np.zeros([N+1,N+1])
  h = 1/N
  for i in range(N+1):
    for j in range(N+1):
      if i==0 or i==N or j==0 or j==N:
        lh_matrix[i,j]=0
      else:
        lh_matrix[i,j]=(h**2/2) * (((i)/N)**2 + ((j)/N)**2 - (i)/N - (j)/N - 3/2)
  return lh_matrix

In [6]:
#Test l(N)
print("When N = 8, the lh matrix is:", l(8))

When N = 8, the lh matrix is: [[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.         -0.01342773 -0.01403809 -0.0144043  -0.01452637 -0.0144043
  -0.01403809 -0.01342773  0.        ]
 [ 0.         -0.01403809 -0.01464844 -0.01501465 -0.01513672 -0.01501465
  -0.01464844 -0.01403809  0.        ]
 [ 0.         -0.0144043  -0.01501465 -0.01538086 -0.01550293 -0.01538086
  -0.01501465 -0.0144043   0.        ]
 [ 0.         -0.01452637 -0.01513672 -0.01550293 -0.015625   -0.01550293
  -0.01513672 -0.01452637  0.        ]
 [ 0.         -0.0144043  -0.01501465 -0.01538086 -0.01550293 -0.01538086
  -0.01501465 -0.0144043   0.        ]
 [ 0.         -0.01403809 -0.01464844 -0.01501465 -0.01513672 -0.01501465
  -0.01464844 -0.01403809  0.        ]
 [ 0.         -0.01342773 -0.01403809 -0.0144043  -0.01452637 -0.0144043
  -0.01403809 -0.01342773  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.     

In [0]:
def initial_PDE_value(N):
  u = np.zeros([N+1,N+1])
  for i in range(N+1):
    for j in range(N+1):
      if i==0 or i==N or j==0 or j==N:
        u[i, j]=((i/N-1/2)**2 + (j/N-1/2)**2)
      else:
        u[i, j] = 0
  return u

In [8]:
#Test initial PDE value
print("The initial vaule of the PDE is:", initial_PDE_value(8))

The initial vaule of the PDE is: [[0.5      0.390625 0.3125   0.265625 0.25     0.265625 0.3125   0.390625
  0.5     ]
 [0.390625 0.       0.       0.       0.       0.       0.       0.
  0.390625]
 [0.3125   0.       0.       0.       0.       0.       0.       0.
  0.3125  ]
 [0.265625 0.       0.       0.       0.       0.       0.       0.
  0.265625]
 [0.25     0.       0.       0.       0.       0.       0.       0.
  0.25    ]
 [0.265625 0.       0.       0.       0.       0.       0.       0.
  0.265625]
 [0.3125   0.       0.       0.       0.       0.       0.       0.
  0.3125  ]
 [0.390625 0.       0.       0.       0.       0.       0.       0.
  0.390625]
 [0.5      0.390625 0.3125   0.265625 0.25     0.265625 0.3125   0.390625
  0.5     ]]


In [0]:
def F(N,u):
  g = gamma(N)
  lh = l(N)
  h = 1/N
  v = np.zeros([N+1,N+1])
  for i in range(N+1):
    for j in range(N+1):
      if i==0 or i==N or j==0 or j==N:
        v[i,j]=u[i,j]
      else:
        v[i,j]=g*(lh[i,j] + (1/4) * (u[i+1,j]+u[i,j+1]+u[i-1,j]+u[i,j-1]))
  return v

In [11]:
#Test interation function
print("The outcome is:", F(8, initial_PDE_value(8)))

The outcome is: [[ 0.5         0.390625    0.3125      0.265625    0.25        0.265625
   0.3125      0.390625    0.5       ]
 [ 0.390625    0.18047481  0.06359012  0.05159884  0.04760174  0.05159884
   0.06359012  0.18047481  0.390625  ]
 [ 0.3125      0.06359012 -0.01453488 -0.01489826 -0.01501938 -0.01489826
  -0.01453488  0.06359012  0.3125    ]
 [ 0.265625    0.05159884 -0.01489826 -0.01526163 -0.01538275 -0.01526163
  -0.01489826  0.05159884  0.265625  ]
 [ 0.25        0.04760174 -0.01501938 -0.01538275 -0.01550388 -0.01538275
  -0.01501938  0.04760174  0.25      ]
 [ 0.265625    0.05159884 -0.01489826 -0.01526163 -0.01538275 -0.01526163
  -0.01489826  0.05159884  0.265625  ]
 [ 0.3125      0.06359012 -0.01453488 -0.01489826 -0.01501938 -0.01489826
  -0.01453488  0.06359012  0.3125    ]
 [ 0.390625    0.18047481  0.06359012  0.05159884  0.04760174  0.05159884
   0.06359012  0.18047481  0.390625  ]
 [ 0.5         0.390625    0.3125      0.265625    0.25        0.265625
   0.3125 

In [0]:
# Define value iteration function
def value_iteration(eps_hat,N):
  v = initial_PDE_value(N)
  flag = 1
  n = 0
  while flag:
    n+=1
    u=v
    v=F(N,u)
    eps=np.max(np.abs(u-v))
    if eps<eps_hat:
      flag=0
  return [eps,n,v]

In [13]:
#Test value interation function
print("The outcome is:", value_iteration(0.0001, 8))

The outcome is: [9.14354904360086e-05, 58, array([[ 0.5       ,  0.390625  ,  0.3125    ,  0.265625  ,  0.25      ,
         0.265625  ,  0.3125    ,  0.390625  ,  0.5       ],
       [ 0.390625  ,  0.28111535,  0.20288037,  0.15592491,  0.14027904,
         0.15592491,  0.20288037,  0.28111535,  0.390625  ],
       [ 0.3125    ,  0.20288037,  0.12454026,  0.0775344 ,  0.06184983,
         0.0775344 ,  0.12454026,  0.20288037,  0.3125    ],
       [ 0.265625  ,  0.15592491,  0.0775344 ,  0.03046517,  0.01478977,
         0.03046517,  0.0775344 ,  0.15592491,  0.265625  ],
       [ 0.25      ,  0.14027904,  0.06184983,  0.01478977, -0.00091948,
         0.01478977,  0.06184983,  0.14027904,  0.25      ],
       [ 0.265625  ,  0.15592491,  0.0775344 ,  0.03046517,  0.01478977,
         0.03046517,  0.0775344 ,  0.15592491,  0.265625  ],
       [ 0.3125    ,  0.20288037,  0.12454026,  0.0775344 ,  0.06184983,
         0.0775344 ,  0.12454026,  0.20288037,  0.3125    ],
       [ 0.390625  

In [0]:
# Define the exact solution of PDE
def exact_PDE_value(N):
  v_exact_value=np.zeros([N+1,N+1])
  for i in range(N+1):
    for j in range(N+1):
      v_exact_value[i,j]=(i/N-1/2)**2 + (j/N-1/2)**2
  return v_exact_value

In [16]:
#Test exact solution of PDE
print("The outcome is:" , exact_PDE_value(8))

The outcome is: [[0.5      0.390625 0.3125   0.265625 0.25     0.265625 0.3125   0.390625
  0.5     ]
 [0.390625 0.28125  0.203125 0.15625  0.140625 0.15625  0.203125 0.28125
  0.390625]
 [0.3125   0.203125 0.125    0.078125 0.0625   0.078125 0.125    0.203125
  0.3125  ]
 [0.265625 0.15625  0.078125 0.03125  0.015625 0.03125  0.078125 0.15625
  0.265625]
 [0.25     0.140625 0.0625   0.015625 0.       0.015625 0.0625   0.140625
  0.25    ]
 [0.265625 0.15625  0.078125 0.03125  0.015625 0.03125  0.078125 0.15625
  0.265625]
 [0.3125   0.203125 0.125    0.078125 0.0625   0.078125 0.125    0.203125
  0.3125  ]
 [0.390625 0.28125  0.203125 0.15625  0.140625 0.15625  0.203125 0.28125
  0.390625]
 [0.5      0.390625 0.3125   0.265625 0.25     0.265625 0.3125   0.390625
  0.5     ]]


In [17]:
error = np.max(np.abs(value_iteration(0.0001,2**3)[2]-exact_PDE_value(2**3)))
print("When h=",2**3,", the maxnorm of error is:", error)

When h= 8 , the maxnorm of error is: 0.0009194811677614096
